In [1]:
import gensim
from gensim.models.doc2vec import Doc2Vec

In [2]:
#加载PV_DBOW模型
pv_dbow_model = Doc2Vec.load('./PV_DBOW_model')

## 输入数据

In [18]:
#读取训练文本
import pandas as pd
df = pd.read_csv('./data/YNU.EDU2018-ScenicWord/train_first.csv')
df.head(5)

,Id,Discuss,Score
0,201e8bf2-77a2-3a98-9fcf-4ce03914e712,好大的一个游乐公园，已经去了2次，但感觉还没有玩够似的！会有第三，第四次的,5
1,f4d51947-eac4-3005-9d3c-2f32d6068a2d,新中国成立也是在这举行，对我们中国人来说有些重要及深刻的意义！,4
2,74aa7ae4-03a4-394c-bee0-5702d3a3082a,庐山瀑布非常有名，也有非常多个瀑布，只是最好看的非三叠泉莫属，推荐一去,4
3,099661c2-4360-3c49-a2fe-8c783764f7db,个人觉得颐和园是北京最值的一起的地方，不过相比下门票也是最贵的，比起故宫的雄伟与气势磅礴，颐...,5
4,97ca672d-e558-3542-ba7b-ee719bba1bab,迪斯尼一日游,5


In [19]:
#文本分词
#对字符串数据content分词
import re
import jieba
def fenci_content(content):
    seg_list = []
    addlist = ['\n',',',"\n"]
    text = re.sub(u'[a-zA-Z0-9.。：:;；，,)）(（！!?？”“\"\<\>《》/]','',content).split()  
    for t in text:
        seg = list(jieba.cut(t))
        for s in seg:
            if s not in addlist and len(s)>1:
                seg_list.append(s)
    return seg_list

In [20]:
from sklearn.cross_validation import train_test_split
import numpy
#将数据划分成训练数据和测试数据
train,test = train_test_split(df)

In [7]:
#读取待预测的数据
import pandas as pd
df_predict = pd.read_csv('./data/YNU.EDU2018-ScenicWord/predict_first.csv')
df_predict.head(5)

,Id,Discuss
0,9a1caf96-681e-3c11-b588-43ac742d7fd2,快乐之旅
1,82b450db-65c2-351c-84fb-761d76582680,岛上看日落的地方，视野很开阔，非常漂亮
2,2eec4606-590c-3fa2-b846-7f92441c54a6,很有鲁迅风味 很喜欢这样有文化的地方
3,509f9a68-ac41-35ff-9d2e-2fc12f73ed7f,去乌鲁木齐还能不去天山天池吗，哈哈哈～
4,395f4b22-1c5f-328a-a19d-5065e0530cbc,非常满意，直接拿身份证刷机入园就行了，不用排队买票，比较节约时间


In [5]:
import gensim 
import collections
import multiprocessing
#标签，doc2vec要求训练数据附上标签
def tokenize1(data,tokens_only=False):
    data_len = len(data)
    corpus = []
    for i in range(data_len):
        d = data.iloc[i]
        content = d.Discuss.decode('utf8')
        if tokens_only:
            corpus.append(fenci_content(content))
        else:
            #对训练数据打标签
            corpus.append(gensim.models.doc2vec.TaggedDocument(fenci_content(content),[i]))
    return corpus

In [7]:
train_corpus = list(tokenize1(train))
test_corpus = list(tokenize1(test,tokens_only=True))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.514 seconds.
Prefix dict has been built succesfully.


In [8]:
predict_corpus = list(tokenize1(df_predict,tokens_only=True))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.333 seconds.
Prefix dict has been built succesfully.


## doc向量生成

## 训练/测试文本

In [9]:
#新输入文本（词的list形式）,生成文本向量
#model.infer_vector(['only','you','can','prevent','forecast','fires'])
import time

t0 = time.time()
#分别统计训练数据集和测试集的长度
train_le = len(train_corpus)
test_le = len(test_corpus)

columns = [i for i in range(80)]
feature = columns[:]
columns.append('score')

pd_train = pd.DataFrame(columns = columns)
pd_test = pd.DataFrame(columns = columns)

#以pd.DataFrame构造训练集
for i in range(train_le):
    content = train_corpus[i]
    t1 = time.time()
    vec = pv_dbow_model.infer_vector(content.words)
    t2 = time.time()
    #print 'infer_vector need :',t2-t1
    score = train.iloc[i].Score
    featurelist = vec.tolist()
    featurelist.append(score)
    row = pd.DataFrame([featurelist],columns=columns)
    #print 'row:',row
    pd_train = pd_train.append(row,ignore_index=True)
    #print pd_train.head(5)
    if i%500 == -1%500:  
        print 'train processing %s'%i

#以pd.DataFrame构造测试集
for i in range(test_le):
    content = test_corpus[i]
    vec = pv_dbow_model.infer_vector(content)
    score = test.iloc[i].Score
    featurelist = vec.tolist()
    featurelist.append(score)
    row = pd.DataFrame([featurelist],columns=columns)
    pd_test = pd_test.append(row,ignore_index=True)
    if i%500 == -1%500:  
        print 'test processing %s'%i

t3 = time.time()
print 'total time :',t3-t0

train processing 499
train processing 999
train processing 1499
train processing 1999
train processing 2499
train processing 2999
train processing 3499
train processing 3999
train processing 4499
train processing 4999
train processing 5499
train processing 5999
train processing 6499
train processing 6999
train processing 7499
train processing 7999
train processing 8499
train processing 8999
train processing 9499
train processing 9999
train processing 10499
train processing 10999
train processing 11499
train processing 11999
train processing 12499
train processing 12999
train processing 13499
train processing 13999
train processing 14499
train processing 14999
train processing 15499
train processing 15999
train processing 16499
train processing 16999
train processing 17499
train processing 17999
train processing 18499
train processing 18999
train processing 19499
train processing 19999
train processing 20499
train processing 20999
train processing 21499
train processing 21999
train proc

## 预测文本

In [9]:
#预测数据转换成doc vec
predict_le = len(predict_corpus)

columns = [i for i in range(80)]
feature = columns[:]

pd_predict = pd.DataFrame(columns = columns)

#以pd.DataFrame构造预测集
for i in range(predict_le):
    content = predict_corpus[i]
    #t1 = time.time()
    vec = pv_dbow_model.infer_vector(content)
    #t2 = time.time()
    #print 'infer_vector need :',t2-t1
    #score = train.iloc[i].Score
    featurelist = vec.tolist()
    #featurelist.append(score)
    row = pd.DataFrame([featurelist],columns=columns)
    #print 'row:',row
    pd_predict = pd_predict.append(row,ignore_index=True)
    #print pd_train.head(5)
    if i%500 == -1%500:  
        print 'train processing %s'%i

train processing 499
train processing 999
train processing 1499
train processing 1999
train processing 2499
train processing 2999
train processing 3499
train processing 3999
train processing 4499
train processing 4999
train processing 5499
train processing 5999
train processing 6499
train processing 6999
train processing 7499
train processing 7999
train processing 8499
train processing 8999
train processing 9499
train processing 9999
train processing 10499
train processing 10999
train processing 11499
train processing 11999
train processing 12499
train processing 12999
train processing 13499
train processing 13999
train processing 14499
train processing 14999
train processing 15499
train processing 15999
train processing 16499
train processing 16999
train processing 17499
train processing 17999
train processing 18499
train processing 18999
train processing 19499
train processing 19999
train processing 20499
train processing 20999
train processing 21499
train processing 21999
train proc

In [15]:
#保存数据
pd_train.to_csv('./data/pd_train.csv',index=False)
pd_test.to_csv('./data/pd_test.csv',index=False)
pd_predict.to_csv('./data/pd_predict.csv',index=False)